In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
from transformers import AutoConfig, AutoTokenizer, AutoModel, TFAutoModel

model_name_or_path = "HooshvareLab/bert-fa-zwnj-base"
config = AutoConfig.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

model = AutoModel.from_pretrained(model_name_or_path)



In [4]:
!pip install hazm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 37.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 70.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not in

In [5]:
pip install cleantext

Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

import hazm

import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import tqdm

import os
import re
import json
import copy
import collections

In [7]:
data1 = pd.read_csv('/kaggle/input/snappfood-persian-sentiment-analysis/Snappfood - Sentiment Analysis.csv', sep='\t')
data2 = pd.read_csv('/kaggle/input/digikala-comments-persian-sentiment-analysis/data.csv')
data3 = pd.read_csv('/kaggle/input/taaghche/taghche.csv')

In [8]:
data2 = data2[data2['Suggestion'].isin([1, 3])]

In [9]:
data2.head()


,Text,Score,Suggestion
0,این اولین تجربه من برای خرید ایفون هست امروز...,100,1
1,خرید این محصول رو توصیه میکنم,84,1
2,1 ساله این گوشی رو دارم هیچ نقطه ضعفی ازش ند...,60,1
3,سلام خدمت دوستان این گوشی از همه نظر عالی کیف...,96,1
4,سلام دوستانی که نگران شکستن صفحه نمایش هستند ا...,92,1


In [10]:
data2 = data2.rename(columns={"Text": "comment", "Suggestion": "label_id"})

In [11]:
data2.drop('Score', axis=1, inplace=True)

In [12]:
data3.head()

,date,comment,bookname,rate,bookID,like
0,1395/11/14,اسم کتاب No one writes to the Colonel\nترجمش...,سرهنگ کسی ندارد برایش نامه بنویسد,0.0,3.0,2.0
1,1395/11/14,"طاقچه عزیز،نام کتاب""کسی به سرهنگ نامه نمینویسد...",سرهنگ کسی ندارد برایش نامه بنویسد,5.0,3.0,2.0
2,1394/06/06,بنظرم این اثر مارکز خیلی از صد سال تنهایی که ب...,سرهنگ کسی ندارد برایش نامه بنویسد,5.0,3.0,0.0
3,1393/09/02,به نظر کتاب خوبی میومد اما من از ترجمش خوشم نی...,سرهنگ کسی ندارد برایش نامه بنویسد,2.0,3.0,0.0
4,1393/06/29,کتاب خوبی است,سرهنگ کسی ندارد برایش نامه بنویسد,3.0,3.0,0.0


In [13]:
def rate_to_label(val):
    if val>= 3:
        return 1
    else:
        return 0

In [14]:
data3['rate'] = data3['rate'].apply(rate_to_label)

In [15]:
data3.drop(['date', 'bookname', 'bookID', 'like'], axis=1, inplace=True)

In [16]:
data3 = data3.rename(columns={"rate": "label_id"})

In [17]:
data = pd.concat([data1, data2, data3], ignore_index=True).drop(['label', 'Unnamed: 0'], axis=1)

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142671 entries, 0 to 142670
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   comment   142650 non-null  object 
 1   label_id  142151 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.2+ MB


In [19]:
data = data.dropna()

In [20]:
data['comment_len_by_words'] = data['comment'].apply(lambda t: len(hazm.word_tokenize(t)))


In [22]:
# data.drop('Unnamed: 0', axis=1, inplace=True)

In [23]:
data

,comment,label_id,comment_len_by_words
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1.0,9
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0.0,28
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1.0,19
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0.0,21
4,شیرینی وانیلی فقط یک مدل بود.,0.0,7
...,...,...,...
142666,من چاپیش رو خوندم خیلی هم لذت بردم.به نظرم موض...,1.0,54
142667,کاش یه تخفیف میذاشتن . خیلی دوست دارم بخونمش,1.0,9
142668,این کتاب داستان درد و رنج کشاورزان بیکارشده‌ی ...,1.0,116
142669,کتابی فوق العاده زیبا و عالی,1.0,6


In [24]:
min_max_len = data["comment_len_by_words"].min(), data["comment_len_by_words"].max()


In [25]:
print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')


Min: 1 	Max: 2691


In [26]:
def data_gl_than(data, less_than=100.0, greater_than=0.0, col='comment_len_by_words'):
    data_length = data[col].values

    data_glt = sum([1 for length in data_length if greater_than < length <= less_than])

    data_glt_rate = (data_glt / len(data_length)) * 100

    print(f'Texts with word length of greater than {greater_than} and less than {less_than} includes {data_glt_rate:.2f}% of the whole!')
     

In [27]:
data_gl_than(data, 256, 3)


Texts with word length of greater than 3 and less than 256 includes 92.23% of the whole!


In [28]:
minlim, maxlim = 3, 256


In [29]:
data['comment_len_by_words'] = data['comment_len_by_words'].apply(lambda len_t: len_t if minlim < len_t <= maxlim else None)
data = data.dropna(subset=['comment_len_by_words'])
data = data.reset_index(drop=True)

In [30]:
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=data['comment_len_by_words']
))

fig.update_layout(
    title_text='Distribution of word counts within comments',
    xaxis_title_text='Word Count',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [31]:
data

,comment,label_id,comment_len_by_words
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1.0,9.0
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0.0,28.0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1.0,19.0
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0.0,21.0
4,شیرینی وانیلی فقط یک مدل بود.,0.0,7.0
...,...,...,...
131079,من چاپیش رو خوندم خیلی هم لذت بردم.به نظرم موض...,1.0,54.0
131080,کاش یه تخفیف میذاشتن . خیلی دوست دارم بخونمش,1.0,9.0
131081,این کتاب داستان درد و رنج کشاورزان بیکارشده‌ی ...,1.0,116.0
131082,کتابی فوق العاده زیبا و عالی,1.0,6.0


In [32]:
!pip install -q clean-text[gpl]

In [33]:
from cleantext import clean


In [34]:
data

,comment,label_id,comment_len_by_words
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1.0,9.0
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0.0,28.0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1.0,19.0
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0.0,21.0
4,شیرینی وانیلی فقط یک مدل بود.,0.0,7.0
...,...,...,...
131079,من چاپیش رو خوندم خیلی هم لذت بردم.به نظرم موض...,1.0,54.0
131080,کاش یه تخفیف میذاشتن . خیلی دوست دارم بخونمش,1.0,9.0
131081,این کتاب داستان درد و رنج کشاورزان بیکارشده‌ی ...,1.0,116.0
131082,کتابی فوق العاده زیبا و عالی,1.0,6.0


In [35]:
comments = ' '.join(snappfood_data['comment'])
char_counts = {}

for char in comments:
    char_counts[char] = char_counts.get(char, 0) + 1

thresh = 30        
char_below_thresh = {char for char, count in char_counts.items() if count < thresh}

def remove_unnecessary_char(comment):
    for char in char_below_thresh:
        comment = comment.replace(char, '')
        
    for char in ['.', '?', '!', ')', '(', '-']:
        comment = comment.replace(char, '')
    
    return comment

data['comment'] = data['comment'].apply(remove_unnecessary_char)


show_samples(snappfood_train['comment'])

In [ ]:
def tokenize(comment):
    hazm_tokenizer = WordTokenizer(replace_numbers=True, replace_ids=True)
    tokens = hazm_tokenizer.tokenize(comment)

    hazm_lemmatizer = Lemmatizer()            
    lemms = [hazm_lemmatizer.lemmatize(token) for token in tokens]

    final_words = [word for word in lemms if word not in stopwords_list()]
    return ' '.join(final_words)
data['comment'] = data['comment'].apply(tokenize)

show_samples(snappfood_train['comment'], 10)

In [ ]:
fasttext_model = FastText(snappfood_data['comment'], min_count=1, workers=4)

In [ ]:
def transform(X, model):
    embed = []
    for row in X:
        each_embed = [] 
        for word in row.split():
            if word in model.wv:
                each_embed.append(model.wv[word])
        embed.append(each_embed)
        
    return embed

In [ ]:
data = transform(data['comment'], fasttext_model)

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131084 entries, 0 to 131083
Data columns (total 3 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   comment               131084 non-null  object 
 1   label_id              131084 non-null  float64
 2   comment_len_by_words  131084 non-null  float64
dtypes: float64(2), object(1)
memory usage: 3.0+ MB


In [38]:
data

,comment,label_id,comment_len_by_words
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1.0,9.0
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0.0,28.0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1.0,19.0
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0.0,21.0
4,شیرینی وانیلی فقط یک مدل بود.,0.0,7.0
...,...,...,...
131079,من چاپیش رو خوندم خیلی هم لذت بردم.به نظرم موض...,1.0,54.0
131080,کاش یه تخفیف میذاشتن . خیلی دوست دارم بخونمش,1.0,9.0
131081,این کتاب داستان درد و رنج کشاورزان بیکارشده‌ی ...,1.0,116.0
131082,کتابی فوق العاده زیبا و عالی,1.0,6.0


In [39]:
# data['cleaned_comment'] = data['comment'].apply(cleaning)


# data['cleaned_comment_len_by_words'] = data['cleaned_comment'].apply(lambda t: len(hazm.word_tokenize(t)))

# data['cleaned_comment_len_by_words'] = data['cleaned_comment_len_by_words'].apply(lambda len_t: len_t if minlim < len_t <= maxlim else len_t)
# data = data.dropna(subset=['cleaned_comment_len_by_words'])
# data = data.reset_index(drop=True)

# data.head()

In [40]:
from hazm import stopwords_list

In [41]:
filtered_word_list = data['comment'][:] #make a copy of the word_list
for word in data['comment']: # iterate over word_list
    if word in stopwords_list():
        filtered_word_list.remove(word) # remove word from filtered_word_list if it is a stopword

In [42]:
# stopwords_list()

In [51]:
labels = list(sorted(data['label_id'].unique()))

In [53]:
negative_data = data[data['label_id'] == 0]
positive_data = data[data['label_id'] == 1]

cutting_point = min(len(negative_data), len(positive_data))

if cutting_point <= len(negative_data):
    negative_data = negative_data.sample(n=cutting_point).reset_index(drop=True)

if cutting_point <= len(positive_data):
    positive_data = positive_data.sample(n=cutting_point).reset_index(drop=True)

new_data = pd.concat([negative_data, positive_data])
new_data = new_data.sample(frac=1).reset_index(drop=True)
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94074 entries, 0 to 94073
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   comment               94074 non-null  object 
 1   label_id              94074 non-null  float64
 2   comment_len_by_words  94074 non-null  float64
dtypes: float64(2), object(1)
memory usage: 2.2+ MB


In [54]:
new_data['label_id'] = new_data['label_id'].apply(lambda t: labels.index(t))

train, test = train_test_split(new_data, test_size=0.1, random_state=1, stratify=new_data['label_id'])
train, valid = train_test_split(train, test_size=0.1, random_state=1, stratify=train['label_id'])

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

x_train, y_train = train['comment'].values.tolist(), train['label_id'].values.tolist()
x_valid, y_valid = valid['comment'].values.tolist(), valid['label_id'].values.tolist()
x_test, y_test = test['comment'].values.tolist(), test['label_id'].values.tolist()

print(train.shape)
print(valid.shape)
print(test.shape)

(76199, 3)
(8467, 3)
(9408, 3)


In [55]:
from transformers import BertConfig, BertTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

import torch
import torch.nn as nn
import torch.nn.functional as F

In [56]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device: cuda:0
CUDA is available!  Training on GPU ...


In [57]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16

EPOCHS = 1
EEVERY_EPOCH = 1180
LEARNING_RATE = 2e-5
CLIP = 0.0

MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
OUTPUT_PATH = '/kaggle/working/pytorch_model.bin'

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
     

In [58]:


label2id = {label: i for i, label in enumerate(labels)}
id2label = {v: k for k, v in label2id.items()}

print(f'label2id: {label2id}')
print(f'id2label: {id2label}')

label2id: {0.0: 0, 1.0: 1, 3.0: 2}
id2label: {0: 0.0, 1: 1.0, 2: 3.0}


In [59]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH, **{
        'label2id': label2id,
        'id2label': id2label,
    })

print(config.to_json_string())

vocab.txt:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

{
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": 0.0,
    "1": 1.0,
    "2": 3.0
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "0.0": 0,
    "1.0": 1,
    "3.0": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.37.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}



In [60]:
idx = np.random.randint(0, len(train))
sample_comment = train.iloc[idx]['comment']
sample_label = train.iloc[idx]['label_id']

print(f'Sample: \n{sample_comment}\n{sample_label}')

Sample: 
اصلا نفهمیدم این همه تعریف از این کتاب برای چیه!
نمیدونمم چرا نویسنده انقدر همه کاراکترای فیلمو انقدر پست و مزخرف گذاشته!
ی آدم درست و حسابی نداشت😒
1


In [61]:
tokens = tokenizer.tokenize(sample_comment)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f'  Comment: {sample_comment}')
print(f'   Tokens: {tokenizer.convert_tokens_to_string(tokens)}')
print(f'Token IDs: {token_ids}')

  Comment: اصلا نفهمیدم این همه تعریف از این کتاب برای چیه!
نمیدونمم چرا نویسنده انقدر همه کاراکترای فیلمو انقدر پست و مزخرف گذاشته!
ی آدم درست و حسابی نداشت😒
   Tokens: اصلا نفهمیدم این همه تعریف از این کتاب برای چیه ! نمیدونمم چرا نویسنده انقدر همه کاراکترای فیلمو انقدر پست و مزخرف گذاشته ! ی ادم درست و حسابی [UNK]
Token IDs: [5899, 49777, 2802, 3218, 4568, 2791, 2802, 3250, 2831, 21628, 1001, 31746, 2015, 3660, 5317, 6853, 3218, 9925, 2784, 3169, 2005, 6853, 4997, 1379, 43139, 5323, 1001, 1442, 5983, 4242, 1379, 10423, 1]


In [62]:
encoding = tokenizer.encode_plus(
    sample_comment,
    max_length=32,
    truncation=True,
    add_special_tokens=True, 
    return_token_type_ids=True,
    return_attention_mask=True,
    padding='max_length',
    return_tensors='pt', 
)

print(f'Keys: {encoding.keys()}\n')
for k in encoding.keys():
    print(f'{k}:\n{encoding[k]}')

Keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

input_ids:
tensor([[    2,  5899, 49777,  2802,  3218,  4568,  2791,  2802,  3250,  2831,
         21628,  1001, 31746,  2015,  3660,  5317,  6853,  3218,  9925,  2784,
          3169,  2005,  6853,  4997,  1379, 43139,  5323,  1001,  1442,  5983,
          4242,     4]])
token_type_ids:
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]])


In [63]:
class Dataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset for Taaghche. """

    def __init__(self, tokenizer, comments, targets=None, label_list=None, max_len=128):
        self.comments = comments
        self.targets = targets
        self.has_target = isinstance(targets, list) or isinstance(targets, np.ndarray)

        self.tokenizer = tokenizer
        self.max_len = max_len

        
        self.label_map = {label: i for i, label in enumerate(label_list)} if isinstance(label_list, list) else {}
    
    def __len__(self):
        return len(self.comments)

    def __getitem__(self, item):
        comment = str(self.comments[item])

        if self.has_target:
            target = self.label_map.get(str(self.targets[item]), str(self.targets[item]))

        encoding = self.tokenizer.encode_plus(
            comment,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')
        
        inputs = {
            'comment': comment,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }

        if self.has_target:

            inputs['targets'] = torch.tensor(int(target), dtype=torch.long)
        
        return inputs


def create_data_loader(x, y, tokenizer, max_len, batch_size, label_list):
    dataset = Dataset(
        comments=x,
        targets=y,
        tokenizer=tokenizer,
        max_len=max_len, 
        label_list=label_list)
    
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [64]:
label_list = [0, 1]
train_data_loader = create_data_loader(train['comment'].to_numpy(), train['label_id'].to_numpy(), tokenizer, MAX_LEN, TRAIN_BATCH_SIZE, label_list)
valid_data_loader = create_data_loader(valid['comment'].to_numpy(), valid['label_id'].to_numpy(), tokenizer, MAX_LEN, VALID_BATCH_SIZE, label_list)
test_data_loader = create_data_loader(test['comment'].to_numpy(), None, tokenizer, MAX_LEN, TEST_BATCH_SIZE, label_list)

In [66]:
sample_data = next(iter(train_data_loader))

print(sample_data.keys())

print(sample_data['comment'])
print(sample_data['input_ids'].shape)
print(sample_data['input_ids'][0, :])
print(sample_data['attention_mask'].shape)
print(sample_data['attention_mask'][0, :])
print(sample_data['token_type_ids'].shape)
print(sample_data['token_type_ids'][0, :])
print(sample_data['targets'].shape)
print(sample_data['targets'][0])

dict_keys(['comment', 'input_ids', 'attention_mask', 'token_type_ids', 'targets'])
['Kheili ba harekatesh hal kardam yelitri bud to yek onim rikht e bud goft male to', 'کتاب نسبتا خوبی بود، فقط یه سری نکاتش با فرهنگ ما نمی خونه.\nاما در کل خوب بود', 'زوربا نشون می ده چقد راحت میشه از زندگی لذت برد', 'خیلی عالی فقط مثل عکسش سیب زمینی نداشت', 'قلمی روان، روایتی به غایت تأثیرگذار، ترجمه خوب، پرداختن به جزئیات در حد قابل قبول و داستانی گیرا از این کتاب، یک اثر ستودنی ساختن\nاز قضا همین پرداختن به جزئیاته که داستان رو جذاب\u200cتر کرده و تونسته از دل یک سرگذشت دردناک، یک رمان کم\u200cنظیر بیرون بکشه\nسانسوری هم اگر به اجبار انجام شده، به روند داستان لطمه\u200cای وارد نکرده و خواننده صفر تا صد ماجرا رو متوجه میشه\nخوندنش رو به اهالی طاقچه توصیه می\u200cکنم👌', 'یکی از ضعیف ترین کارهای خون اشامی.\nتوضیحات زیست شناسی زیاد و خسته کننده،ترجمه ی نه چندان خوشایند.من توصیه نمیکنم', 'خیلی عالی\u200cبود همه چیز هم زود بدستم رسید هم داغ و نان تازه و هم کیفیت عالی', 'هر دفعه تعریف کردم، ولی ایندفعه اصلا

In [69]:
class SentimentModel(nn.Module):

    def __init__(self, config):
        super(SentimentModel, self).__init__()

        self.bert = BertModel.from_pretrained(MODEL_NAME_OR_PATH)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, 1)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        pooled_output= self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids)
#         print(pooled_output[0])
        pooled_output = self.dropout(pooled_output[0])
        logits = self.classifier(pooled_output)
        return logits 

In [70]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()
pt_model = None

!nvidia-smi

Mon Feb 19 08:13:39 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0              26W / 250W |      2MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [71]:
pt_model = SentimentModel(config=config)
pt_model = pt_model.to(device)

print('pt_model', type(pt_model))

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

pt_model <class '__main__.SentimentModel'>


In [72]:
sample_data_comment = sample_data['comment']
sample_data_input_ids = sample_data['input_ids']
sample_data_attention_mask = sample_data['attention_mask']
sample_data_token_type_ids = sample_data['token_type_ids']
sample_data_targets = sample_data['targets']

sample_data_input_ids = sample_data_input_ids.to(device)
sample_data_attention_mask = sample_data_attention_mask.to(device)
sample_data_token_type_ids = sample_data_token_type_ids.to(device)
sample_data_targets = sample_data_targets.to(device)




outputs = pt_model(sample_data_input_ids, sample_data_attention_mask, sample_data_token_type_ids)
_, preds = torch.max(outputs, dim=1)

# print(outputs[:5, :])
# print(preds[:5])

In [73]:
def simple_accuracy(y_true, y_pred):
    return (y_true == y_pred).mean()

def acc_and_f1(y_true, y_pred, average='weighted'):
    acc = simple_accuracy(y_true, y_pred)
    f1 = f1_score(y_true=y_true, y_pred=y_pred, average=average)
    return {
        "acc": acc,
        "f1": f1,
    }

def y_loss(y_true, y_pred, losses):
    y_true = torch.stack(y_true).cpu().detach().numpy()
    y_pred = torch.stack(y_pred).cpu().detach().numpy()
    y = [y_true, y_pred]
    loss = np.mean(losses)

    return y, loss


def eval_op(model, data_loader, loss_fn):
    model.eval()

    losses = []
    y_pred = []
    y_true = []

    with torch.no_grad():
        for dl in tqdm(data_loader, total=len(data_loader), desc="Evaluation... "):
            
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']
            targets = dl['targets']

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            targets = targets.to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
            
            _, preds = torch.max(outputs, dim=1)

            loss = loss_fn(outputs.squeeze(), targets)

            losses.append(loss.item())

            y_pred.extend(preds)
            y_true.extend(targets)
    
    eval_y, eval_loss = y_loss(y_true, y_pred, losses)
    return eval_y, eval_loss


def train_op(model, 
             data_loader, 
             loss_fn, 
             optimizer, 
             scheduler, 
             step=0, 
             print_every_step=100, 
             eval=False,
             eval_cb=None,
             eval_loss_min=np.Inf,
             eval_data_loader=None, 
             clip=0.0):
    
    model.train()

    losses = []
    y_pred = []
    y_true = []

    for dl in tqdm(data_loader, total=len(data_loader), desc="Training... "):
        step += 1

        input_ids = dl['input_ids']
        attention_mask = dl['attention_mask']
        token_type_ids = dl['token_type_ids']
        targets = dl['targets']

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)
        
        _, preds = torch.max(outputs, dim=1)
#         print(outputs)
#         print(targets)
        loss = loss_fn(outputs.squeeze(), targets.to(device))
        
        losses.append(loss.item())

        loss.backward()

#         if clip > 0.0:
#             nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)

        optimizer.step()

        scheduler.step()

        y_pred.extend(preds)
        y_true.extend(targets)
        if step % 1180 == 0:
            train_y, train_loss = y_loss(y_true, y_pred, losses)
            train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

            if step % print_every_step == 0:
                eval_y, eval_loss = eval_op(model, eval_data_loader, loss_fn)
                eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

                if hasattr(eval_cb, '__call__'):
                    eval_loss_min = eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min)

    train_y, train_loss = y_loss(y_true, y_pred, losses)

    return train_y, train_loss, step, eval_loss_min

In [ ]:
optimizer = AdamW(pt_model.parameters(), lr=LEARNING_RATE, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

step = 0
eval_loss_min = np.Inf
history = collections.defaultdict(list)


def eval_callback(epoch, epochs, output_path):
    def eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min):
        statement = ''
        statement += 'Epoch: {}/{}...'.format(epoch, epochs)
        statement += 'Step: {}...'.format(step)
        
        statement += 'Train Loss: {:.6f}...'.format(train_loss)
        statement += 'Train Acc: {:.3f}...'.format(train_score['acc'])

        statement += 'Valid Loss: {:.6f}...'.format(eval_loss)
        statement += 'Valid Acc: {:.3f}...'.format(eval_score['acc'])

        print(statement)

        if eval_loss <= eval_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                eval_loss_min,
                eval_loss))
            
            torch.save(model.state_dict(), output_path)
            eval_loss_min = eval_loss
        
        return eval_loss_min


    return eval_cb


for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
    train_y, train_loss, step, eval_loss_min = train_op(
        model=pt_model, 
        data_loader=train_data_loader, 
        loss_fn=loss_fn, 
        optimizer=optimizer, 
        scheduler=scheduler, 
        step=step, 
        print_every_step=EEVERY_EPOCH, 
        eval=True,
        eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
        eval_loss_min=eval_loss_min,
        eval_data_loader=valid_data_loader, 
        clip=CLIP)
    
    train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')
    
    eval_y, eval_loss = eval_op(
        model=pt_model, 
        data_loader=valid_data_loader, 
        loss_fn=loss_fn)
    
    eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')
    
    history['train_acc'].append(train_score['acc'])
    history['train_loss'].append(train_loss)
    history['val_acc'].append(eval_score['acc'])
    history['val_loss'].append(eval_loss)

In [75]:
def predict(model, comments, tokenizer, max_len=128, batch_size=32):
    data_loader = create_data_loader(comments, None, tokenizer, max_len, batch_size, None)
    
    predictions = []
    prediction_probs = []

    
    model.eval()
    with torch.no_grad():
        for dl in tqdm(data_loader, position=0):
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
            
            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds)
            prediction_probs.extend(F.softmax(outputs, dim=1))

    predictions = torch.stack(predictions).cpu().detach().numpy()
    prediction_probs = torch.stack(prediction_probs).cpu().detach().numpy()

    return predictions, prediction_probs
     

test_comments = test['comment'].to_numpy()
preds, probs = predict(pt_model, test_comments, tokenizer, max_len=128)

print(preds.shape, probs.shape)

  0%|          | 0/294 [00:00<?, ?it/s]

(9408, 1) (9408, 128, 1)


In [78]:
y_test, y_pred = [label_list.index(label) for label in test['label_id'].values], preds

print(f'F1: {f1_score(y_test, y_pred, average="weighted")}')
print()
print(classification_report(y_test, y_pred))

F1: 0.824348594354243

              precision    recall  f1-score   support

           0       0.85      0.79      0.82      4704
           1       0.80      0.86      0.83      4704

    accuracy                           0.82      9408
   macro avg       0.83      0.82      0.82      9408
weighted avg       0.83      0.82      0.82      9408



In [82]:
concatenated = pd.concat([test, pd.DataFrame(y_pred)], axis="columns")
concatenated = concatenated.rename(columns={0: "predicted"})

In [106]:
concatenated[concatenated['label_id'] != concatenated['predicted']].to_excel('wrongs.xlsx')